In [53]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from d2l import torch as d2l
import numpy as np
import pandas as pd
from ptflops import get_model_complexity_info
from pathlib import Path
import subprocess
import os
import time
from pynvml import *
import threading
import queue

### Path

In [54]:
'''find the Model path'''
# find the current path
current_path = os.getcwd()
print('The current path is:', current_path)

# find the parent path
parent_path = Path(current_path).parent
print('The parent path is:', parent_path)

# find the data path
data_path = parent_path / '3080/Test_data'
print('The data path is:', data_path)

The current path is: /home/GreenAI/3080
The parent path is: /home/GreenAI
The data path is: /home/GreenAI/3080/Test_data


### Model

In [55]:
'''
def alexnet(img_channel, num_labels):
    net = nn.Sequential(
        # 这里使用一个11*11的更大窗口来捕捉对象。
        # 同时，步幅为4，以减少输出的高度和宽度。
        # 另外，输出通道的数目远大于LeNet
        nn.Conv2d(img_channel, 64, kernel_size=11, stride=4, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
        nn.Conv2d(64, 192, kernel_size=5, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 使用三个连续的卷积层和较小的卷积窗口。
        # 除了最后的卷积层，输出通道的数量进一步增加。
        # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
        nn.Conv2d(192, 384, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.AdaptiveAvgPool2d((6, 6)),   # 使用全局平均池化对每个通道中所有元素求平均并直接将结果传递到全连接层
        nn.Flatten(),
        # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
        nn.Linear(256 * 6 * 6, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
        nn.Linear(4096, num_labels))
    return net
'''

'\ndef alexnet(img_channel, num_labels):\n    net = nn.Sequential(\n        # 这里使用一个11*11的更大窗口来捕捉对象。\n        # 同时，步幅为4，以减少输出的高度和宽度。\n        # 另外，输出通道的数目远大于LeNet\n        nn.Conv2d(img_channel, 64, kernel_size=11, stride=4, padding=2), nn.ReLU(),\n        nn.MaxPool2d(kernel_size=3, stride=2),\n        # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数\n        nn.Conv2d(64, 192, kernel_size=5, padding=2), nn.ReLU(),\n        nn.MaxPool2d(kernel_size=3, stride=2),\n        # 使用三个连续的卷积层和较小的卷积窗口。\n        # 除了最后的卷积层，输出通道的数量进一步增加。\n        # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度\n        nn.Conv2d(192, 384, kernel_size=3, padding=1), nn.ReLU(),\n        nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),\n        nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(),\n        nn.MaxPool2d(kernel_size=3, stride=2),\n        nn.AdaptiveAvgPool2d((6, 6)),   # 使用全局平均池化对每个通道中所有元素求平均并直接将结果传递到全连接层\n        nn.Flatten(),\n        # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合\n        nn.Linear(256 * 6 * 6, 4096)

In [56]:
class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
    
    
def resnet(img_channel, num_labels):
    # blk = Residual(3,6, use_1x1conv=True, strides=2)

    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                    nn.BatchNorm2d(64), nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    def resnet_block(input_channels, num_channels, num_residuals,
                    first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(input_channels, num_channels,
                                    use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(num_channels, num_channels))
        return blk

    b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
    b3 = nn.Sequential(*resnet_block(64, 128, 2))
    b4 = nn.Sequential(*resnet_block(128, 256, 2))
    b5 = nn.Sequential(*resnet_block(256, 512, 2))

    net = nn.Sequential(b1, b2, b3, b4, b5,
                        nn.AdaptiveAvgPool2d((1,1)),
                        nn.Flatten(), nn.Linear(512, num_labels))
    return net

In [57]:
# print the model structure
# net = alexnet(1, 10)    
net = resnet(1, 10)
# print(net)
# print each layer
layer_name = []
for layer in net:
    name = layer.__class__.__name__
    layer_name.append(name)
# find the unique layer name, and fix the order
layer_name = sorted(list(set(layer_name)))
print('The layer name is:', layer_name)
# the number of layers, which contains ['AdaptiveAvgPool2d', 'Conv2d', 'Dropout', 'Flatten', 'Linear', 'MaxPool2d', 'ReLU']
num_layers = len(layer_name) 
print('The number of layers is:', num_layers)

The layer name is: ['AdaptiveAvgPool2d', 'Flatten', 'Linear', 'Sequential']
The number of layers is: 4


In [58]:
# # print the model structure
# AlexNet = alexnet(1, 10)    
# # print(net)
# # print each layer
# alexlayer = []
# DNNlayer = []
# num = 0
# for layer in AlexNet:
#     num += 1
#     name = layer.__class__.__name__
#     layer_name = name[:4] + str(num)
#     DNNlayer.append(name)
#     alexlayer.append(layer_name)
# # find the unique layer name, and fix the order
# DNNlayer_org = sorted(list(set(DNNlayer)))
# print('The layer name is:', alexlayer)
# print('The layer name after orged is:', DNNlayer_org)
# # the number of layers, which contains ['AdaptiveAvgPool2d', 'Conv2d', 'Dropout', 'Flatten', 'Linear', 'MaxPool2d', 'ReLU']
# alexlayer_num = len(alexlayer) 
# print('The number of layers is:', alexlayer_num)

In [59]:
net = resnet(1, 10)    
LayerName = []
block_num = 0
resblock_num = 0

for num, layer in net.named_children():  # 使用 named_children 来获取层名和层
    layername = layer.__class__.__name__
    print(f"{num}: {layername}")  # 打印层名和层类名
    if layer.__class__.__name__ == 'Sequential':
        block_num += 1
        for sublayernum, sublayer in layer.named_children():  # 再次使用 named_children
            sublayername = sublayer.__class__.__name__
            if sublayername == 'Residual':
                resblock_num += 1
            print(f"  {sublayernum}: {sublayername}")
            layer_label = f'{layername[0]}{num}_{sublayername[0]}{sublayernum}'
            LayerName.append(layer_label)  # 收集子块的类型
    else:
        layer_label = f'{layername[0]}{num}'
        LayerName.append(layer_label)
            
print('The layer name is:', LayerName)
print(f'The length of layer name is: {len(LayerName)}')
print('The number of blocks is:', block_num)
print('The number of inception blocks is:', resblock_num)

0: Sequential
  0: Conv2d
  1: BatchNorm2d
  2: ReLU
  3: MaxPool2d
1: Sequential
  0: Residual
  1: Residual
2: Sequential
  0: Residual
  1: Residual
3: Sequential
  0: Residual
  1: Residual
4: Sequential
  0: Residual
  1: Residual
5: AdaptiveAvgPool2d
6: Flatten
7: Linear
The layer name is: ['S0_C0', 'S0_B1', 'S0_R2', 'S0_M3', 'S1_R0', 'S1_R1', 'S2_R0', 'S2_R1', 'S3_R0', 'S3_R1', 'S4_R0', 'S4_R1', 'A5', 'F6', 'L7']
The length of layer name is: 15
The number of blocks is: 5
The number of inception blocks is: 8


build different alexnet model for different datasets

In [60]:
# 对于不同的数据集，要设置不同的img_channel和num_labels
# Fashion-MNIST中的图像通道数为1，类别数为10
# alexnet_f = alexnet(1, 10)
resnet_f = resnet(1, 10)
'''
# CIFAR100中的图像通道数为3，类别数为100
alexnet_c = alexnet(3, 100)
# CIFAR10中的图像通道数为3，类别数为10
alexnet_c10 = alexnet(3, 10)
'''

'\n# CIFAR100中的图像通道数为3，类别数为100\nalexnet_c = alexnet(3, 100)\n# CIFAR10中的图像通道数为3，类别数为10\nalexnet_c10 = alexnet(3, 10)\n'

In [61]:
# # fashion mnist
# with torch.cuda.device(0):
#     macs_f, params_f = get_model_complexity_info(alexnet_f, (1, 224, 224), as_strings=True,
#                                             print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))

# fashion mnist
with torch.cuda.device(0):
    macs_f, params_f = get_model_complexity_info(resnet_f, (1, 224, 224), as_strings=True,
                                            print_per_layer_stat=True, verbose=True)
    print('{:<30}  {:<8}'.format('Computational complexity: ', macs_f))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params_f))
'''
print('*'*50)

# cifar100
with torch.cuda.device(0):
    macs_c, params_c = get_model_complexity_info(alexnet_c, (3, 224, 224), as_strings=True,
                                            print_per_layer_stat=True, verbose=True)
    print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))
    
# cifar10
with torch.cuda.device(0):
    macs_c10, params_c10 = get_model_complexity_info(alexnet_c10, (3, 224, 224), as_strings=True,
                                            print_per_layer_stat=True, verbose=True)
    print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c10))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params_c10))
'''

Sequential(
  11.18 M, 100.000% Params, 1.74 GMac, 99.820% MACs, 
  (0): Sequential(
    3.33 k, 0.030% Params, 43.35 MMac, 2.482% MACs, 
    (0): Conv2d(3.2 k, 0.029% Params, 40.14 MMac, 2.298% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(128, 0.001% Params, 1.61 MMac, 0.092% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(0, 0.000% Params, 802.82 KMac, 0.046% MACs, )
    (3): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.046% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    148.22 k, 1.326% Params, 464.83 MMac, 26.611% MACs, 
    (0): Residual(
      74.11 k, 0.663% Params, 232.42 MMac, 13.306% MACs, 
      (conv1): Conv2d(36.93 k, 0.330% Params, 115.81 MMac, 6.630% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(36.93 k, 0.330% Params, 115.81 MMac, 6.630% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 

"\nprint('*'*50)\n\n# cifar100\nwith torch.cuda.device(0):\n    macs_c, params_c = get_model_complexity_info(alexnet_c, (3, 224, 224), as_strings=True,\n                                            print_per_layer_stat=True, verbose=True)\n    print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c))\n    print('{:<30}  {:<8}'.format('Number of parameters: ', params_c))\n    \n# cifar10\nwith torch.cuda.device(0):\n    macs_c10, params_c10 = get_model_complexity_info(alexnet_c10, (3, 224, 224), as_strings=True,\n                                            print_per_layer_stat=True, verbose=True)\n    print('{:<30}  {:<8}'.format('Computational complexity: ', macs_c10))\n    print('{:<30}  {:<8}'.format('Number of parameters: ', params_c10))\n"

### Datasets

In [62]:
X_f = torch.randn(size=(1, 1, 224, 224), dtype=torch.float32) # fashion mnist
'''
X_c = torch.randn(size=(1, 3, 224, 224), dtype=torch.float32) # cifar100
x_c10 = torch.randn(size=(1, 3, 224, 224), dtype=torch.float32) # cifar10
'''

# for layer in alexnet_f:
#     X_f=layer(X_f)
#     print(layer.__class__.__name__,'output shape:\t',X_f.shape)


for layer in resnet_f:
    X_f=layer(X_f)
    print(layer.__class__.__name__,'output shape:\t',X_f.shape)

'''
print('*'*50)

for layer in alexnet_c:
    X_c=layer(X_c)
    print(layer.__class__.__name__,'output shape:\t',X_c.shape)
    
print('*'*50)

for layer in alexnet_c10:
    x_c10=layer(x_c10)
    print(layer.__class__.__name__,'output shape:\t',x_c10.shape)
'''

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


"\nprint('*'*50)\n\nfor layer in alexnet_c:\n    X_c=layer(X_c)\n    print(layer.__class__.__name__,'output shape:\t',X_c.shape)\n    \nprint('*'*50)\n\nfor layer in alexnet_c10:\n    x_c10=layer(x_c10)\n    print(layer.__class__.__name__,'output shape:\t',x_c10.shape)\n"

In [63]:
# load the data
# fashion mnist
def get_dataloader_workers():
    """Use 4 processes to read the data.

    Defined in :numref:`sec_utils`"""
    return 4

def load_data_fashion_mnist(batch_size, resize=None):
    """下载Fashion-MNIST数据集，然后将其加载到内存中

    Defined in :numref:`sec_fashion_mnist`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=get_dataloader_workers()))
'''
def load_data_cifar100(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory.

    Defined in :numref:`sec_utils`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    # import the cifar100 dataset
    cifar_train = torchvision.datasets.CIFAR100(
        root="../data", train=True, transform=trans, download=True)
    cifar_test = torchvision.datasets.CIFAR100(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(cifar_train, batch_size, shuffle=True,
                                        num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(cifar_test, batch_size, shuffle=False,
                                        num_workers=get_dataloader_workers()))
    
def load_data_cifar10(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory.

    Defined in :numref:`sec_utils`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    # import the cifar100 dataset
    cifar_train = torchvision.datasets.CIFAR10(
        root="../data", train=True, transform=trans, download=True)
    cifar_test = torchvision.datasets.CIFAR10(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(cifar_train, batch_size, shuffle=True,
                                        num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(cifar_test, batch_size, shuffle=False,
                                        num_workers=get_dataloader_workers()))
'''
    

'\ndef load_data_cifar100(batch_size, resize=None):\n    """Download the Fashion-MNIST dataset and then load it into memory.\n\n    Defined in :numref:`sec_utils`"""\n    trans = [transforms.ToTensor()]\n    if resize:\n        trans.insert(0, transforms.Resize(resize))\n    trans = transforms.Compose(trans)\n    # import the cifar100 dataset\n    cifar_train = torchvision.datasets.CIFAR100(\n        root="../data", train=True, transform=trans, download=True)\n    cifar_test = torchvision.datasets.CIFAR100(\n        root="../data", train=False, transform=trans, download=True)\n    return (torch.utils.data.DataLoader(cifar_train, batch_size, shuffle=True,\n                                        num_workers=get_dataloader_workers()),\n            torch.utils.data.DataLoader(cifar_test, batch_size, shuffle=False,\n                                        num_workers=get_dataloader_workers()))\n    \ndef load_data_cifar10(batch_size, resize=None):\n    """Download the Fashion-MNIST dataset

### Parameters

In [64]:
batch_size = [256]
# epochs = [10, 20, 30, 40, 50, 80, 100]
epochs = [1]
rounds = 1

### Timer

### Train Model

In [65]:
'''
def train_func(net, train_iter, test_iter, alexlayer, num_epochs, lr, device):
    def init_weights(m): # 初始化权重
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    # create a ndarray to store each layer's total running time of each epoch
    Layers_time = np.zeros((len(alexlayer), num_epochs)) # each row is a layer, each column is an epoch
    print(f'The name of the layers are: {alexlayer}')
    Train_part_time = np.zeros((6, num_epochs)) # store the time to device, forward and backward time, and test time of each epoch
    Train_time = np.zeros(num_epochs) # store the total training time of each epoch
    Train_acc = np.zeros(num_epochs) # store the training accuracy of each epoch
    Test_acc = np.zeros(num_epochs) # store the test accuracy of each epoch
    Epoch_time = np.zeros(num_epochs) # store the total time of each epoch
    Epoch_energy = np.zeros((num_epochs,1), dtype='object') # store the total energy of each epoch
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    timer = d2l.Timer()
    train_timer = d2l.Timer()
    ttd_timer = d2l.Timer()
    forward_timer = d2l.Timer()
    loss_timer = d2l.Timer()
    backward_timer = d2l.Timer()
    opt_timer = d2l.Timer()
    layer_timer = d2l.Timer()
    test_timer = d2l.Timer()
    # start training
    for epoch in range(num_epochs):
        print('The epoch is:', epoch+1)
        timer.start()
        net.train()
        train_epoch, ttd_epoch, forward_epoch, loss_epoch, backward_epoch, opt_epoch, testtime_epoch= 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        layer_epoch = np.zeros((len(alexlayer), 1)) # store the total running time of each layer in one epoch
        metric = d2l.Accumulator(3)  # train_loss, train_acc, num_examples   
        # start the nvidia-smi command
        with open('gpu_power_usage.csv', 'w') as file:
            # Start the nvidia-smi command
            nvidia_smi_process = subprocess.Popen(
                ["nvidia-smi", "--query-gpu=power.draw", "--format=csv", "--loop-ms=10"],
                stdout=file,  # Redirect the output directly to the file
                stderr=subprocess.PIPE,
                text=True)
        train_timer.start()
        for i, (X, y) in enumerate(train_iter):
            lnamenum = 0
            print('The batch is:', i+1)
            optimizer.zero_grad()
            # to device
            torch.cuda.synchronize()  # 等待数据传输完成
            ttd_timer.start()
            # sleep for 2 seconds
            time.sleep(2)
            X, y = X.to(device), y.to(device)
            torch.cuda.synchronize()  # 等待数据传输完成
            # sleep for 2 seconds
            time.sleep(2)
            ttd_epoch += ttd_timer.stop()
            # forward
            forward_timer.start()
            y_hat = X
            for layer in net:
                name = layer.__class__.__name__ # 获取层的名字
                lnamenum += 1
                lname = name[:4] + str(lnamenum)
                layer_index = alexlayer.index(lname)
                layer_timer.start()
                y_hat = layer(y_hat)
                torch.cuda.synchronize()  # 等待数据传输完成
                layer_epoch[layer_index] += layer_timer.stop()
            torch.cuda.synchronize()  # 等待数据传输完成
            forward_epoch += forward_timer.stop()
            # loss
            loss_timer.start()
            l = loss_fn(y_hat, y)
            # backward
            torch.cuda.synchronize()  # 等待数据传输完成
            loss_epoch += loss_timer.stop()
            backward_timer.start()
            l.backward()
            torch.cuda.synchronize()  # 等待数据传输完成
            backward_epoch += backward_timer.stop()
            # optimize
            opt_timer.start()   
            optimizer.step()
            torch.cuda.synchronize()  # 等待数据传输完成
            opt_epoch += opt_timer.stop()
            with torch.no_grad():
                metric.add(l*X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_acc = metric[1] / metric[2]
        train_epoch = train_timer.stop()
        test_timer.start()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        testtime_epoch = test_timer.stop()
        print(f'train acc {train_acc:.3f}, test acc {test_acc:.3f}')
        print('epoch %d, time %f sec' % (epoch+1, timer.sum()))
        # store the time and acc data
        Epoch_time[epoch] = timer.stop()
        print(f'The total time of the {epoch} is:', Epoch_time[epoch])
        Layers_time[:, epoch] = layer_epoch.flatten()
        Train_part_time[:, epoch] = ttd_epoch, forward_epoch, loss_epoch, backward_epoch, opt_epoch, testtime_epoch
        Train_time[epoch] = train_epoch
        Train_acc[epoch] = train_acc
        Test_acc[epoch] = test_acc
        # stop the nvidia-smi command
        nvidia_smi_process.terminate()
        # calculate the energy consumption of each epoch
        GPU_df = pd.read_csv('gpu_power_usage.csv')
        for row in range(len(GPU_df)):
            GPU_df.iloc[row,0] = GPU_df.iloc[row,0].replace(' W','')
        Consumption_df = GPU_df.astype(float)  
        EnergyDatai = Consumption_df.iloc[:,0].values # 将数据转换为numpy数组
        # store the energy data
        Epoch_energy[epoch,0] = EnergyDatai
    return Layers_time, Train_part_time, Train_time, Train_acc, Test_acc, Epoch_time, Epoch_energy
'''

'\ndef train_func(net, train_iter, test_iter, alexlayer, num_epochs, lr, device):\n    def init_weights(m): # 初始化权重\n        if type(m) == nn.Linear or type(m) == nn.Conv2d:\n            nn.init.xavier_uniform_(m.weight)\n    net.apply(init_weights)\n    print(\'training on\', device)\n    net.to(device)\n    # create a ndarray to store each layer\'s total running time of each epoch\n    Layers_time = np.zeros((len(alexlayer), num_epochs)) # each row is a layer, each column is an epoch\n    print(f\'The name of the layers are: {alexlayer}\')\n    Train_part_time = np.zeros((6, num_epochs)) # store the time to device, forward and backward time, and test time of each epoch\n    Train_time = np.zeros(num_epochs) # store the total training time of each epoch\n    Train_acc = np.zeros(num_epochs) # store the training accuracy of each epoch\n    Test_acc = np.zeros(num_epochs) # store the test accuracy of each epoch\n    Epoch_time = np.zeros(num_epochs) # store the total time of each epoch\

In [66]:
def nvml_sampling_thread(handle, data_queue, stop_event, sampling_interval=0.01):
    """
    在单独的线程中定期调用 NVML, 获取功耗数据并存储到 data_queue 中。
    参数：
    - handle: nvmlDeviceGetHandleByIndex(0) 得到的 GPU 句柄
    - data_queue: 用于存放 (timestamp, power_in_watts) 数据的队列
    - stop_event: 当此事件被设置时，线程应结束循环
    - sampling_interval: 采样间隔（秒）
    """
    while not stop_event.is_set():
        current_time = time.time()
        current_power = nvmlDeviceGetPowerUsage(handle) / 1000.0  # mW -> W
        data_queue.put((current_time, current_power))
        time.sleep(sampling_interval)

In [67]:
'''
def train_func(net, train_iter, test_iter, alexlayer, num_epochs, lr, device):
    def init_weights(m): # 初始化权重
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    # create a ndarray to store each layer's total running time of each epoch
    # Layers_time = np.zeros((len(alexlayer), num_epochs)) # each row is a layer, each column is an epoch
    print(f'The name of the layers are: {alexlayer}')
    # Train_part_time = np.zeros((6, num_epochs)) # store the time to device, forward and backward time, and test time of each epoch
    # Train_time = np.zeros(num_epochs) # store the total training time of each epoch
    # Epoch_time = np.zeros(num_epochs) # store the total time of each epoch
    # Epoch_energy = np.zeros((num_epochs,1), dtype='object') # store the total energy of each epoch
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    timer = d2l.Timer()
    # train_timer = d2l.Timer()
    ttd_timer = d2l.Timer()
    # forward_timer = d2l.Timer()
    # loss_timer = d2l.Timer()
    # backward_timer = d2l.Timer()
    # opt_timer = d2l.Timer()
    # layer_timer = d2l.Timer()
    # test_timer = d2l.Timer()
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    # 创建一个队列来存储功率数据
    power_data_queue = queue.Queue()

    # 创建一个线程停止事件
    stop_event = threading.Event()

    # 启动采样线程，每10ms获取一次功耗数据
    sampling_interval = 0.005

    sampler_thread1 = threading.Thread(target=nvml_sampling_thread, args=(handle, power_data_queue, stop_event, sampling_interval))
    start_time = time.time()
    ttd_total = []
    # start training
    for epoch in range(num_epochs):
        ttd_epoch = 0.0
        print('The epoch is:', epoch+1)
        timer.start()
        net.train()
        # train_epoch, forward_epoch, loss_epoch, backward_epoch, opt_epoch, testtime_epoch= 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        # layer_epoch = np.zeros((len(alexlayer), 1)) # store the total running time of each layer in one epoch
        metric = d2l.Accumulator(3)  # train_loss, train_acc, num_examples
        ##########################################################################################################   
        # start the nvidia-smi command
        # with open('gpu_power_usage.csv', 'w') as file:
        #     # Start the nvidia-smi command
        #     nvidia_smi_process = subprocess.Popen(
        #         ["nvidia-smi", "--query-gpu=power.draw", "--format=csv", "--loop-ms=10"],
        #         stdout=file,  # Redirect the output directly to the file
        #         stderr=subprocess.PIPE,
        #         text=True)
        ##########################################################################################################
        # train_timer.start()
        for i, (X, y) in enumerate(train_iter):
            # lnamenum = 0
            print('The batch is:', i+1)
            optimizer.zero_grad()
            # to device
            torch.cuda.synchronize()  # 等待数据传输完成
            ttd_timer.start()
            sampler_thread1.start()
            # start_power = nvmlDeviceGetPowerUsage(handle) / 1000.0
            X, y = X.to(device), y.to(device)
            torch.cuda.synchronize()  # 等待数据传输完成
            duration = ttd_timer.stop()
            # 训练结束后，通知线程停止并等待其结束
            stop_event.set()
            sampler_thread1.join()
            # end_power = nvmlDeviceGetPowerUsage(handle) / 1000.0
            # avg_power = (start_power + end_power) / 2.0
            # energy_joules = avg_power * duration
            # print(f"Data transfer took {duration:.6f}s, Estimated Energy: {energy_joules:.6f}J")
            # forward
            # forward_timer.start()
            y_hat = net(X)
            # for layer in net:
            #     name = layer.__class__.__name__ # 获取层的名字
            #     lnamenum += 1
            #     lname = name[:4] + str(lnamenum)
            #     layer_index = alexlayer.index(lname)
            #     layer_timer.start()
            #     y_hat = layer(y_hat)
            #     torch.cuda.synchronize()  # 等待数据传输完成
            #     layer_epoch[layer_index] += layer_timer.stop()
            torch.cuda.synchronize()  # 等待数据传输完成
            # forward_epoch += forward_timer.stop()
            # loss
            # loss_timer.start()
            l = loss_fn(y_hat, y)
            # backward
            torch.cuda.synchronize()  # 等待数据传输完成
            # loss_epoch += loss_timer.stop()
            # backward_timer.start()
            l.backward()
            torch.cuda.synchronize()  # 等待数据传输完成
            # backward_epoch += backward_timer.stop()
            # optimize
            # opt_timer.start()   
            optimizer.step()
            torch.cuda.synchronize()  # 等待数据传输完成
            # opt_epoch += opt_timer.stop()
            with torch.no_grad():
                metric.add(l*X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_acc = metric[1] / metric[2]
        # train_epoch = train_timer.stop()
        # test_timer.start()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        # testtime_epoch = test_timer.stop()
        print(f'train acc {train_acc:.3f}, test acc {test_acc:.3f}')
        print('epoch %d, time %f sec' % (epoch+1, timer.sum()))
        # store the time and acc data
        # Epoch_time[epoch] = timer.stop()
        # print(f'The total time of the {epoch} is:', Epoch_time[epoch])
        # Layers_time[:, epoch] = layer_epoch.flatten()
        # Train_part_time[:, epoch] = ttd_epoch, forward_epoch, loss_epoch, backward_epoch, opt_epoch, testtime_epoch
        # Train_time[epoch] = train_epoch
        # Train_acc[epoch] = train_acc
        # Test_acc[epoch] = test_acc
        # stop the nvidia-smi command
        # nvidia_smi_process.terminate()
        # calculate the energy consumption of each epoch
        # GPU_df = pd.read_csv('gpu_power_usage.csv')
        # for row in range(len(GPU_df)):
        #     GPU_df.iloc[row,0] = GPU_df.iloc[row,0].replace(' W','')
        # Consumption_df = GPU_df.astype(float)  
        # EnergyDatai = Consumption_df.iloc[:,0].values # 将数据转换为numpy数组
        # store the energy data
        # Epoch_energy[epoch,0] = EnergyDatai
    # 关闭 NVML
    nvmlShutdown()
    # return Layers_time, Train_part_time, Train_time, Epoch_time
'''

'\ndef train_func(net, train_iter, test_iter, alexlayer, num_epochs, lr, device):\n    def init_weights(m): # 初始化权重\n        if type(m) == nn.Linear or type(m) == nn.Conv2d:\n            nn.init.xavier_uniform_(m.weight)\n    net.apply(init_weights)\n    print(\'training on\', device)\n    net.to(device)\n    # create a ndarray to store each layer\'s total running time of each epoch\n    # Layers_time = np.zeros((len(alexlayer), num_epochs)) # each row is a layer, each column is an epoch\n    print(f\'The name of the layers are: {alexlayer}\')\n    # Train_part_time = np.zeros((6, num_epochs)) # store the time to device, forward and backward time, and test time of each epoch\n    # Train_time = np.zeros(num_epochs) # store the total training time of each epoch\n    # Epoch_time = np.zeros(num_epochs) # store the total time of each epoch\n    # Epoch_energy = np.zeros((num_epochs,1), dtype=\'object\') # store the total energy of each epoch\n    optimizer = torch.optim.SGD(net.parameters

In [68]:
def nvml_sampling_thread(handle, data_queue, stop_event, sampling_interval=0.01):
    while not stop_event.is_set():
        current_time = time.time()
        current_power = nvmlDeviceGetPowerUsage(handle) / 1000.0
        data_queue.put((current_time, current_power))
        time.sleep(sampling_interval)

# def integrate_power_over_interval(samples, start_time, end_time):
#     # 对 [start_time, end_time] 区间内的功率样本进行积分（简单线性近似）
#     # samples 是 (time, power) 的列表，假定按时间排序
#     # 过滤出在 start_time 之前和 end_time 之后的样本
#     filtered = [(t, p) for t, p in samples if start_time <= t <= end_time]
#     if len(filtered) < 2:
#         # 样本过少，无法良好估计，这时可以尝试从samples中找到距离start和end最近的点
#         # 简化处理：返回0或尝试扩展为插值
#         return 0.0

def integrate_power_over_interval(samples, start_time, end_time):
    # 假定 samples是按时间升序排序的 (t, p)
    # 若未排序，请先排序:
    # samples = sorted(samples, key=lambda x: x[0])
    
    def interpolate(samples, target_time):
        # 在 samples 中找到 target_time 左右最近的两个点，并进行线性插值
        # 若 target_time 恰好等于某个样本点时间，直接返回该点功率
        # 若无法找到两侧点（如 target_time在样本时间轴外），根据情况返回None或边界点
        n = len(samples)
        if n == 0:
            return None
        # 若 target_time 小于第一个样本点时间，无法向左插值，这里直接返回第一个点的功率值(或None)
        if target_time <= samples[0][0]:
            # 简化处理：返回最早样本点的功率（或None）
            return samples[0][1]
        # 若 target_time 大于最后一个样本点时间，无法向右插值，返回最后一个点的功率（或None）
        if target_time >= samples[-1][0]:
            return samples[-1][1]

        # 否则，在中间插值
        # 使用二分查找快速定位
        import bisect
        times = [t for t, _ in samples]
        pos = bisect.bisect_left(times, target_time)
        # pos是使times保持有序插入target_time的位置
        # 因为target_time不在已有样本点中，pos不会越界且pos>0且pos<n
        t1, p1 = samples[pos-1]
        t2, p2 = samples[pos]
        # 线性插值： p = p1 + (p2 - p1)*((target_time - t1)/(t2 - t1))
        ratio = (target_time - t1) / (t2 - t1)
        p = p1 + (p2 - p1)*ratio
        return p

    # 从原始 samples 中筛选出位于[start_time, end_time]内的点
    filtered = [(t, p) for t, p in samples if start_time <= t <= end_time]

    # 如果不足2个点，则尝试使用插值
    if len(filtered) < 2:
        # 无论如何都需要在边界处插值出两个点(起码start和end)
        start_power = interpolate(samples, start_time)
        end_power = interpolate(samples, end_time)

        # 如果从样本中无法插值出任何有意义的点（比如samples为空或无法插值），返回0.0
        if start_power is None or end_power is None:
            return 0.0

        # 将插值的边界点加入到 filtered
        # 注意：如果filtered中有一个点在区间内，我们也需要确保边界有两点以上
        # 例如filtered只有一个点在中间，则需要在start和end插值点全部加入。
        # 若filtered为空，则只用start/end两点插值点求积分
        new_filtered = [(start_time, start_power)] + filtered + [(end_time, end_power)]
        # 确保按时间排序
        new_filtered.sort(key=lambda x: x[0])
        filtered = new_filtered

    # 正常积分计算
    if len(filtered) < 2:
        # 经过插值仍不够，返回0
        return 0.0

    total_energy = 0.0
    for i in range(len(filtered)-1):
        t1, p1 = filtered[i]
        t2, p2 = filtered[i+1]
        dt = t2 - t1
        avg_p = (p1 + p2)/2.0
        total_energy += avg_p * dt

    return total_energy
    

# def train_func(net, train_iter, test_iter, alexlayer, num_epochs, lr, device):
def train_func(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    # print(f'The name of the layers are: {alexlayer}')
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    
    # 初始化NVML和采样线程
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    power_data_queue = queue.Queue()
    stop_event = threading.Event()
    sampling_interval = 0.005
    sampler_thread = threading.Thread(target=nvml_sampling_thread, args=(handle, power_data_queue, stop_event, sampling_interval))
    sampler_thread.start()

    for epoch in range(num_epochs):
        print('The epoch is:', epoch+1)
        metric = d2l.Accumulator(3)  # train_loss, train_acc, num_examples
        to_device_intervals = []  # 用来记录本epoch每个batch的to_device时间段
        forward_intervals = []  # 用来记录本epoch每个batch的forward时间段
        backward_intervals = []  # 

        to_device_time_consump_total = 0
        forward_time_consump_total = 0
        backward_time_consump_total = 0

        to_device_energy_consump_total = 0
        forward_energy_consump_total = 0
        backward_energy_consump_total = 0
        

        # 记录epoch开始时队列中已有的数据条数，用于后面区分本epoch的数据
        initial_queue_size = power_data_queue.qsize()

        net.train()
        for i, (X, y) in enumerate(train_iter):
            print('The batch is:', i+1)
            optimizer.zero_grad()
            
            # 记录to_device前后的时间戳
            start_ttd_time = time.time()
            X, y = X.to(device), y.to(device)
            torch.cuda.synchronize()
            end_ttd_time = time.time()
            to_device_intervals.append((start_ttd_time, end_ttd_time))
            ttd_time_consump = end_ttd_time - start_ttd_time
            to_device_time_consump_total += ttd_time_consump

            # time.sleep(0.5)
            # forward
            start_forward_time = time.time()
            y_hat = net(X)
            torch.cuda.synchronize()
            end_forward_time = time.time()
            forward_intervals.append((start_forward_time, end_forward_time))
            forward_time_consump = end_forward_time - start_forward_time
            forward_time_consump_total += forward_time_consump

            l = loss_fn(y_hat, y)

            torch.cuda.synchronize()
            start_backward_time = time.time()
            l.backward()
            torch.cuda.synchronize()
            end_backward_time = time.time()
            backward_intervals.append((start_backward_time, end_backward_time))
            backward_time_consump = end_backward_time - start_backward_time
            backward_time_consump_total += backward_time_consump

            optimizer.step()

            with torch.no_grad():
                metric.add(l*X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_acc = metric[1] / metric[2]

        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        print(f'train acc {train_acc:.3f}, test acc {test_acc:.3f}')

        # 从队列中取出本epoch采样数据
        epoch_samples = []
        # 这里取出initial_queue_size之后加入的所有数据
        total_samples_in_epoch = power_data_queue.qsize() - initial_queue_size
        for _ in range(total_samples_in_epoch):
            epoch_samples.append(power_data_queue.get())

        # 对epoch中每个batch的to_device间隔计算能耗
        for idx, (s_time, e_time) in enumerate(to_device_intervals):
            batch_energy = integrate_power_over_interval(epoch_samples, s_time, e_time)
            to_device_energy_consump_total += batch_energy
            print(f"Epoch {epoch+1}, Batch {idx+1}, to_device Energy: {batch_energy} J")
        
        # 对epoch中每个batch的forward间隔计算能耗
        for idx, (s_time, e_time) in enumerate(forward_intervals):
            batch_energy = integrate_power_over_interval(epoch_samples, s_time, e_time)
            forward_energy_consump_total += batch_energy
            print(f"Epoch {epoch+1}, Batch {idx+1}, forward Energy: {batch_energy} J")

        # 对epoch中每个batch的backward间隔计算能耗
        for idx, (s_time, e_time) in enumerate(backward_intervals):
            batch_energy = integrate_power_over_interval(epoch_samples, s_time, e_time)
            backward_energy_consump_total += batch_energy
            print(f"Epoch {epoch+1}, Batch {idx+1}, backward Energy: {batch_energy} J")
        

        print(f'The total energy consumption in to_device part is: {to_device_energy_consump_total}, and the total time consumed is: {to_device_time_consump_total}.')
        print(f'The total energy consumption in forward part is: {forward_energy_consump_total}, and the total time consumed is: {forward_time_consump_total}.')
        print(f'The total energy consumption in backward part is: {backward_energy_consump_total}, and the total time consumed is: {backward_time_consump_total}')
        # print(f'The total time consumed is: {to_device_time_consump_total}.')
        # print(f'The total time consumed is: {forward_time_consump_total}.')
        # print(f'The total time consumed is: {backward_time_consump_total}')


    # 训练结束后关闭线程
    stop_event.set()
    sampler_thread.join()

    nvmlShutdown()

In [69]:
# # 使用示例
# if __name__ == "__main__":
#     # 初始化 NVML
#     nvmlInit()
#     handle = nvmlDeviceGetHandleByIndex(0)

#     # 创建一个队列来存储功率数据
#     power_data_queue = queue.Queue()

#     # 创建一个线程停止事件
#     stop_event = threading.Event()

#     # 启动采样线程，每10ms获取一次功耗数据
#     sampling_interval = 0.005
#     sampler_thread = threading.Thread(target=nvml_sampling_thread, args=(handle, power_data_queue, stop_event, sampling_interval))
#     sampler_thread.start()

#     # 模拟训练过程
#     # 在真实使用中,这里是你的训练主循环
#     print("Training started...")
#     for epoch in range(3):
#         print(f"Epoch {epoch+1}...")
#         time.sleep(2)  # 模拟训练耗时
#     print("Training finished.")

#     # 训练结束后，通知线程停止并等待其结束
#     stop_event.set()
#     sampler_thread.join()

#     # 获取所有采样数据
#     sampled_data = []
#     while not power_data_queue.empty():
#         sampled_data.append(power_data_queue.get())

#     # 处理数据 (示例：打印前10条数据)
#     print("Sampled power data (first 10):", sampled_data[:10])

#     # 关闭 NVML
#     nvmlShutdown()

### Train the model

In [70]:
def train_model_f(main_folder, batch_size, num_epochs, round, lr, device):
    print(f'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running')
    # create the folder to store the data
    epoch_batch_folder = main_folder/f'E{num_epochs}_B{batch_size}_R{round}'
    # # 判断文件是否存在
    # if epoch_batch_folder.exists():
    #     print("文件存在。")
    # else:
    #     os.makedirs(epoch_batch_folder)
    #     print("文件不存在，已创建。")
    #     print("文件创建于：", epoch_batch_folder)
    train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)
    # show the shape of the data
    list_of_i = []
    for i, (X, y) in enumerate(train_iter):
        if i < 3:
            print('the shape of the', i, 'batch of the train_iter is:', X.shape)
        else:
            pass
        list_of_i.append(i)
    print(f'The number of batches is: {np.array(list_of_i).shape}')
    train_func(resnet_f, train_iter, test_iter, num_epochs, lr, device)
    # Layers_time, Train_part_time, Train_time, Train_acc, Test_acc, \
    #     Epoch_time, Epoch_energy = train_func(resnet_f, train_iter, test_iter, LayerName, block_num, num_epochs, lr, device)
    # # save the data
    # np.save(epoch_batch_folder/'Layers_time.npy', Layers_time)
    # np.save(epoch_batch_folder/'Train_part_time.npy', Train_part_time)
    # np.save(epoch_batch_folder/'Train_time.npy', Train_time)
    # np.save(epoch_batch_folder/'Train_acc.npy', Train_acc)
    # np.save(epoch_batch_folder/'Test_acc.npy', Test_acc)
    # np.save(epoch_batch_folder/'Epoch_time.npy', Epoch_time)
    # np.save(epoch_batch_folder/'Epoch_energy.npy', Epoch_energy)

In [71]:
# def train_model_f(main_folder, batch_size, num_epochs, round, lr, device):
#     print(f'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running')
#     # create the folder to store the data
#     epoch_batch_folder = main_folder/f'E{num_epochs}_B{batch_size}_R{round}'
#     # 判断文件是否存在
#     if epoch_batch_folder.exists():
#         print("文件存在。")
#     else:
#         os.makedirs(epoch_batch_folder)
#         print("文件不存在，已创建。")
#         print("文件创建于：", epoch_batch_folder)
#         train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)
#         # show the shape of the data
#         list_of_i = []
#         for i, (X, y) in enumerate(train_iter):
#             if i < 3:
#                 print('the shape of the', i, 'batch of the train_iter is:', X.shape)
#             else:
#                 pass
#             list_of_i.append(i)
#         print(f'The number of batches is: {np.array(list_of_i).shape}')
#         train_func(alexnet_f, train_iter, test_iter, num_epochs, lr, device)
#         # Layers_time, Train_part_time, Train_time, Train_acc, Test_acc, \
#         #     Epoch_time, Epoch_energy = train_func(alexnet_f, train_iter, test_iter, alexlayer, num_epochs, lr, device)
#         # save the data
#         # np.save(epoch_batch_folder/'Layers_time.npy', Layers_time)
#         # np.save(epoch_batch_folder/'Train_part_time.npy', Train_part_time)
#         # np.save(epoch_batch_folder/'Train_time.npy', Train_time)
#         # np.save(epoch_batch_folder/'Epoch_time.npy', Epoch_time)

In [72]:
'''
def train_model_f(main_folder, batch_size, num_epochs, round, lr, device):
    print(f'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running')
    # create the folder to store the data
    epoch_batch_folder = main_folder/f'E{num_epochs}_B{batch_size}_R{round}'
    # 判断文件是否存在
    if epoch_batch_folder.exists():
        print("文件存在。")
    else:
        os.makedirs(epoch_batch_folder)
        print("文件不存在，已创建。")
        print("文件创建于：", epoch_batch_folder)
        train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)
        # show the shape of the data
        list_of_i = []
        for i, (X, y) in enumerate(train_iter):
            if i < 3:
                print('the shape of the', i, 'batch of the train_iter is:', X.shape)
            else:
                pass
            list_of_i.append(i)
        print(f'The number of batches is: {np.array(list_of_i).shape}')
        Layers_time, Train_part_time, Train_time, Train_acc, Test_acc, \
            Epoch_time, Epoch_energy = train_func(alexnet_f, train_iter, test_iter, alexlayer, num_epochs, lr, device)
        # save the data
        np.save(epoch_batch_folder/'Layers_time.npy', Layers_time)
        np.save(epoch_batch_folder/'Train_part_time.npy', Train_part_time)
        np.save(epoch_batch_folder/'Train_time.npy', Train_time)
        np.save(epoch_batch_folder/'Train_acc.npy', Train_acc)
        np.save(epoch_batch_folder/'Test_acc.npy', Test_acc)
        np.save(epoch_batch_folder/'Epoch_time.npy', Epoch_time)
        np.save(epoch_batch_folder/'Epoch_energy.npy', Epoch_energy)
'''

'\ndef train_model_f(main_folder, batch_size, num_epochs, round, lr, device):\n    print(f\'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running\')\n    # create the folder to store the data\n    epoch_batch_folder = main_folder/f\'E{num_epochs}_B{batch_size}_R{round}\'\n    # 判断文件是否存在\n    if epoch_batch_folder.exists():\n        print("文件存在。")\n    else:\n        os.makedirs(epoch_batch_folder)\n        print("文件不存在，已创建。")\n        print("文件创建于：", epoch_batch_folder)\n        train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)\n        # show the shape of the data\n        list_of_i = []\n        for i, (X, y) in enumerate(train_iter):\n            if i < 3:\n                print(\'the shape of the\', i, \'batch of the train_iter is:\', X.shape)\n            else:\n                pass\n            list_of_i.append(i)\n        print(f\'The number of batches is: {np.array(list_of_i).shape}\')\n        Layers_time, Train_part_time, 

In [73]:
'''
def train_model_c(main_folder, batch_size, num_epochs, round, lr, device):
    print(f'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running')
    # create the folder to store the data
    epoch_batch_folder = main_folder/f'E{num_epochs}_B{batch_size}_R{round}'
    # 判断文件是否存在
    if epoch_batch_folder.exists():
        print("文件存在。")
        pass
    else:
        os.makedirs(epoch_batch_folder)
        print("文件不存在，已创建。")
        print("文件创建于：", epoch_batch_folder)
        train_iter, test_iter = load_data_cifar100(batch_size, resize=224)
        # show the shape of the data
        list_of_i = []
        for i, (X, y) in enumerate(train_iter):
            if i < 3:
                print('the shape of the', i, 'batch of the train_iter is:', X.shape)
            else:
                pass
            list_of_i.append(i)
        print(f'The number of batches is: {np.array(list_of_i).shape}')
        Layers_time, Train_part_time, Train_time, Train_acc, Test_acc, \
            Epoch_time, Epoch_energy = train_func(alexnet_c, train_iter, test_iter, alexlayer, num_epochs, lr, device)
        # save the data
        np.save(epoch_batch_folder/'Layers_time.npy', Layers_time)
        np.save(epoch_batch_folder/'Train_part_time.npy', Train_part_time)
        np.save(epoch_batch_folder/'Train_time.npy', Train_time)
        np.save(epoch_batch_folder/'Train_acc.npy', Train_acc)
        np.save(epoch_batch_folder/'Test_acc.npy', Test_acc)
        np.save(epoch_batch_folder/'Epoch_time.npy', Epoch_time)
        np.save(epoch_batch_folder/'Epoch_energy.npy', Epoch_energy)
'''

'\ndef train_model_c(main_folder, batch_size, num_epochs, round, lr, device):\n    print(f\'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running\')\n    # create the folder to store the data\n    epoch_batch_folder = main_folder/f\'E{num_epochs}_B{batch_size}_R{round}\'\n    # 判断文件是否存在\n    if epoch_batch_folder.exists():\n        print("文件存在。")\n        pass\n    else:\n        os.makedirs(epoch_batch_folder)\n        print("文件不存在，已创建。")\n        print("文件创建于：", epoch_batch_folder)\n        train_iter, test_iter = load_data_cifar100(batch_size, resize=224)\n        # show the shape of the data\n        list_of_i = []\n        for i, (X, y) in enumerate(train_iter):\n            if i < 3:\n                print(\'the shape of the\', i, \'batch of the train_iter is:\', X.shape)\n            else:\n                pass\n            list_of_i.append(i)\n        print(f\'The number of batches is: {np.array(list_of_i).shape}\')\n        Layers_time, Train_pa

In [74]:
'''
def train_model_c10(main_folder, batch_size, num_epochs, round, lr, device):
    print(f'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running')
    # create the folder to store the data
    epoch_batch_folder = main_folder/f'E{num_epochs}_B{batch_size}_R{round}'
    # 判断文件是否存在
    if epoch_batch_folder.exists():
        print("文件存在。")
        pass
    else:
        os.makedirs(epoch_batch_folder)
        print("文件不存在，已创建。")
        print("文件创建于：", epoch_batch_folder)
        train_iter, test_iter = load_data_cifar10(batch_size, resize=224)
        # show the shape of the data
        list_of_i = []
        for i, (X, y) in enumerate(train_iter):
            if i < 3:
                print('the shape of the', i, 'batch of the train_iter is:', X.shape)
            else:
                pass
            list_of_i.append(i)
        print(f'The number of batches is: {np.array(list_of_i).shape}')
        Layers_time, Train_part_time, Train_time, Train_acc, Test_acc, \
            Epoch_time, Epoch_energy = train_func(alexnet_c, train_iter, test_iter, alexlayer, num_epochs, lr, device)
        # save the data
        np.save(epoch_batch_folder/'Layers_time.npy', Layers_time)
        np.save(epoch_batch_folder/'Train_part_time.npy', Train_part_time)
        np.save(epoch_batch_folder/'Train_time.npy', Train_time)
        np.save(epoch_batch_folder/'Train_acc.npy', Train_acc)
        np.save(epoch_batch_folder/'Test_acc.npy', Test_acc)
        np.save(epoch_batch_folder/'Epoch_time.npy', Epoch_time)
        np.save(epoch_batch_folder/'Epoch_energy.npy', Epoch_energy)
'''

'\ndef train_model_c10(main_folder, batch_size, num_epochs, round, lr, device):\n    print(f\'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running\')\n    # create the folder to store the data\n    epoch_batch_folder = main_folder/f\'E{num_epochs}_B{batch_size}_R{round}\'\n    # 判断文件是否存在\n    if epoch_batch_folder.exists():\n        print("文件存在。")\n        pass\n    else:\n        os.makedirs(epoch_batch_folder)\n        print("文件不存在，已创建。")\n        print("文件创建于：", epoch_batch_folder)\n        train_iter, test_iter = load_data_cifar10(batch_size, resize=224)\n        # show the shape of the data\n        list_of_i = []\n        for i, (X, y) in enumerate(train_iter):\n            if i < 3:\n                print(\'the shape of the\', i, \'batch of the train_iter is:\', X.shape)\n            else:\n                pass\n            list_of_i.append(i)\n        print(f\'The number of batches is: {np.array(list_of_i).shape}\')\n        Layers_time, Train_p

In [75]:
lr = 0.01
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('The device is:', device)

The device is: cuda


In [76]:
# create the folder to store the data
main_folder = data_path
print('The folder is:', main_folder)
# find out that if the folder exists in the data path
# 判断文件是否存在
if main_folder.exists():
    print("文件存在。")
else:
    os.makedirs(main_folder)
    print("文件不存在，已创建。")
    print("文件创建于：", main_folder)
for epoch in epochs:
    for batch in batch_size:
        for round in range(rounds):
            train_model_f(main_folder, batch, epoch, round, lr, device)

The folder is: /home/GreenAI/3080/Test_data
文件存在。
The epoch is set: 1, batch is set: 256, is in 1th running
the shape of the 0 batch of the train_iter is: torch.Size([256, 1, 224, 224])
the shape of the 1 batch of the train_iter is: torch.Size([256, 1, 224, 224])
the shape of the 2 batch of the train_iter is: torch.Size([256, 1, 224, 224])


The number of batches is: (235,)
training on cuda
The epoch is: 1
The batch is: 1
The batch is: 2
The batch is: 3
The batch is: 4
The batch is: 5
The batch is: 6
The batch is: 7
The batch is: 8
The batch is: 9
The batch is: 10
The batch is: 11
The batch is: 12
The batch is: 13
The batch is: 14
The batch is: 15
The batch is: 16
The batch is: 17
The batch is: 18
The batch is: 19
The batch is: 20
The batch is: 21
The batch is: 22
The batch is: 23
The batch is: 24
The batch is: 25
The batch is: 26
The batch is: 27
The batch is: 28
The batch is: 29
The batch is: 30
The batch is: 31
The batch is: 32
The batch is: 33
The batch is: 34
The batch is: 35
The batch is: 36
The batch is: 37
The batch is: 38
The batch is: 39
The batch is: 40
The batch is: 41
The batch is: 42
The batch is: 43
The batch is: 44
The batch is: 45
The batch is: 46
The batch is: 47
The batch is: 48
The batch is: 49
The batch is: 50
The batch is: 51
The batch is: 52
The batch is: 53
The batch is: 54
The batch is: 55
The batc

In [77]:
'''
# create the folder to store the data
main_folder = data_path/'cifar100'
print('The folder is:', main_folder)
# find out that if the folder exists in the data path
# 判断文件是否存在
if main_folder.exists():
    print("文件存在。")
else:
    os.makedirs(main_folder)
    print("文件不存在，已创建。")
    print("文件创建于：", main_folder)
for epoch in epochs:
    for batch in batch_size:
        for round in range(rounds):
            train_model_c(main_folder, batch, epoch, round, lr, device)
'''

'\n# create the folder to store the data\nmain_folder = data_path/\'cifar100\'\nprint(\'The folder is:\', main_folder)\n# find out that if the folder exists in the data path\n# 判断文件是否存在\nif main_folder.exists():\n    print("文件存在。")\nelse:\n    os.makedirs(main_folder)\n    print("文件不存在，已创建。")\n    print("文件创建于：", main_folder)\nfor epoch in epochs:\n    for batch in batch_size:\n        for round in range(rounds):\n            train_model_c(main_folder, batch, epoch, round, lr, device)\n'

In [78]:
'''
# create the folder to store the data
main_folder = data_path/'cifar10'
print('The folder is:', main_folder)
# find out that if the folder exists in the data path
# 判断文件是否存在
if main_folder.exists():
    print("文件存在。")
else:
    os.makedirs(main_folder)
    print("文件不存在，已创建。")
    print("文件创建于：", main_folder)
for epoch in epochs:
    for batch in batch_size:
        for round in range(rounds):
            train_model_c10(main_folder, batch, epoch, round, lr, device)
'''

'\n# create the folder to store the data\nmain_folder = data_path/\'cifar10\'\nprint(\'The folder is:\', main_folder)\n# find out that if the folder exists in the data path\n# 判断文件是否存在\nif main_folder.exists():\n    print("文件存在。")\nelse:\n    os.makedirs(main_folder)\n    print("文件不存在，已创建。")\n    print("文件创建于：", main_folder)\nfor epoch in epochs:\n    for batch in batch_size:\n        for round in range(rounds):\n            train_model_c10(main_folder, batch, epoch, round, lr, device)\n'